In [24]:
import cv2
import numpy as np

image = cv2.imread("0_15_hr_png_jpg.rf.10dcf4db9cf5ceaa4277ec2465083fff.jpg")
cv2.imwrite("converted_image.jpg", image)  # Simpan ulang dengan format standar


True

## 1. Object Tracking (YOLO)

In [25]:
import cv2
from ultralytics import YOLO

# Load model YOLO
model = YOLO("best.pt")  # Ganti dengan model Anda

# Prediksi pada gambar
results = model.predict(source="converted_image.jpg", conf=0.5, save=False)

# Baca gambar asli
image = cv2.imread("converted_image.jpg")

# Iterasi bounding box
for i, result in enumerate(results):
    for j, box in enumerate(result.boxes.xyxy):
        x1, y1, x2, y2 = map(int, box)  # Koordinat bounding box
        cropped_image = image[y1:y2, x1:x2]  # Crop area plat nomor

        # Simpan hasil crop untuk debugging
        crop_path = f"cropped_plate_{j}.jpg"
        cv2.imwrite(crop_path, cropped_image)
        print(f"Plat nomor {j} disimpan di {crop_path}")



image 1/1 e:\02_CODE\PYTHON\Kuliah\PCD\Projek Akhir\license plate detection.v1i.yolov8\converted_image.jpg: 800x800 1 LicensePlate, 17.2ms
Speed: 5.3ms preprocess, 17.2ms inference, 2.2ms postprocess per image at shape (1, 3, 800, 800)
Plat nomor 0 disimpan di cropped_plate_0.jpg


## 2. Image Denoising (Weighted Mean Filter)

In [26]:
# Baca gambar
image = cv2.imread("cropped_plate_0.jpg")

# Konversi ke grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Fungsi untuk membuat kernel Gaussian
def gaussian_kernel(size, sigma=1):
   
    kernel = np.zeros((size, size), dtype=np.float32)
    k = size // 2
    for x in range(-k, k + 1):
        for y in range(-k, k + 1):
            kernel[x + k, y + k] = (1 / (2 * np.pi * sigma**2)) * np.exp(-(x**2 + y**2) / (2 * sigma**2))
    return kernel / kernel.sum()

# Fungsi untuk melakukan convolusi
def apply_convolution(image, kernel):
   
    h, w = image.shape
    kh, kw = kernel.shape
    pad_h, pad_w = kh // 2, kw // 2
    
    # Tambahkan padding ke gambar
    padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
    
    # Gambar hasil
    output = np.zeros_like(image, dtype=np.float32)
    
    # Terapkan kernel
    for i in range(h):
        for j in range(w):
            region = padded_image[i:i+kh, j:j+kw]
            output[i, j] = np.sum(region * kernel)
    
    return np.clip(output, 0, 255).astype(np.uint8)

# Baca gambar
image = cv2.imread("cropped_plate_0.jpg")

# Konversi ke grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Buat kernel Gaussian 5x5 dengan sigma=1
gaussian = gaussian_kernel(5, sigma=1)

# Terapkan Gaussian Blur secara manual
blurred_manual = apply_convolution(gray, gaussian)

# Thresholding untuk membuat teks lebih jelas
_, thresh = cv2.threshold(blurred_manual, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Simpan hasil preprocessing
cv2.imwrite("preprocessed_plate_manual.jpg", thresh)


True

## 3. Super Resolusi Citra


In [27]:
# Tingkatkan kontras dengan CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced = clahe.apply(gray)

# Peningkatan resolusi dengan interpolasi bicubic
def upscale_image(image, scale_factor):
    height, width = image.shape[:2]
    new_width = width * scale_factor
    new_height = height * scale_factor
    upscaled = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
    return upscaled

# Terapkan sharpening untuk meningkatkan detail teks
def sharpen_image(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]], dtype=np.float32)
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened

upscaled = upscale_image(enhanced, scale_factor=4)

# Sharpen gambar
sharpened = sharpen_image(upscaled)

# Simpan gambar dengan kontras tinggi
cv2.imwrite("enhanced_plate.jpg", sharpened)


True

In [28]:
import easyocr

# Gunakan EasyOCR

reader = easyocr.Reader(['en'])
text = reader.readtext("enhanced_plate.jpg", detail=0)
print("Teks yang dikenali:", text)


Teks yang dikenali: ['WAE', '920']
